# Préparer les données pour la visualisation

In [2]:
import json
import pandas as pd
import re
from pattern.fr import sentiment
import os
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline

## Charger les données du Raw_data

In [2]:
dBrande={'iphone':'apple','galaxy':'samsung','galaxy s6':'samsung','galaxy s6':'samsung','galaxy s7':'samsung','marvel':'marvel','dc comic':'dc comic','iphhone7plus':'apple'}
def getBrand(s):
    x=re.sub(".json","",s).lower()
    if x in dBrande:
        return dBrande[x]
    return 'NA'

In [16]:
#queries=["iphone","iPhone","IPhone7plus"]
tweetText=[]

for doc in os.listdir(os.getcwd()+"/raw_data"):
    if "json" in doc:
        #doc= "raw_data/subjTweets_"+query+".json"
        with open("raw_data/"+doc, "r") as rFile:
            jsonTweets=rFile.read()
            tweets=json.loads(jsonTweets)
            #s=str(datetime.fromtimestamp(os.stat(os.getcwd()+"/raw_data/"+doc).st_mtime))
            product=re.sub(".json","",doc).lower()
            product=re.sub("tweets_","",product)
            product=re.sub("_[0-9]","",product)
            product=re.sub("dc comics","dc comic",product)
            product=re.sub("dccomics","dc comic",product)
            product=re.sub("samsungg","g",product)
            brand=getBrand(product)
            #t=datetime.fromtimestamp(os.stat(os.getcwd()+"/raw_data/"+doc).st_mtime)
        for tweet in tweets:
            tweetText.append([tweet["text"],product,brand,tweet["created_at"].split()[1:]])

In [49]:
D_mois={'Jan':'1','Fev':'2', 'Mar':'3', 'Avr':'4','Mai':'5', 'Jui':'6', 'Jui':'7', 'Aou':'8', 'Sep':'9', 'Oct':'10', 'Nov':'11', 'Dec':'12' }

In [101]:
def date_format(l):
    m='1'
    try:
        #print(str(l[0]))
        m=str(l[0])
        #print(m)
        if m in D_mois:
            m=D_mois[m]
            #print (type(m))
    except:
        print("error unicode or dictionnary ")
    s=[str(m)]
    s+=[str(l[1])]
    s+=[str(l[-1])]
    k="-".join(s)
    #print(k)
    try:
        return datetime.strptime(k, '%m-%d-%Y')
    except:
        print("error time  ")
        return datetime.strptime('01-01-2000', '%m-%d-%Y')

In [99]:
DataBase = pd.DataFrame(tweetText, columns=['comment','Product','Brand',"Date"])
DataBase['Date']=DataBase['Date'].apply(date_format)
DataBase.head()

## Transformation

### 1. Nettoyer les commentaires

Remplacer les émojis par des adjectifs traduisant leurs significations

In [102]:
D_emojis={r'😰':"peur panique ", r'😉':"heureux malin ",r'😀':"tres heureux ",r'😃':"tres heureux ",r'😄':"tres heureux ", r'😁':"desole ", r'😆':"tres heureux ",r'😅':"tres heureux embarasse ",r'😂':"joie ",r'☺️':"sourire joie ",r'😊':"sourire joie ",
          r'😇':"sourire joie ange ",r'🙂':"content ", r'🙃':"content ",r'😉':"content malin ",r'😌':"indefferent ",r'😍':"amoureux ", r'😘':"bisou aimer ",
          r'😗':"bisou ",r'😙':"bisou ",r'😚':"bisou ",r'😊':"content ",r'😜':"je plaisente ",r'😝':"je plaisente ",r'😛':"je plaisente ",r'🤑':"argent bien aimer ",
          r'🤗':"content ", r'🤓':"nerd ",r'😎':"content ",r'😏':"content malin ",r'😭':"pelurer mauvais ", r'😥':"triste ",r'😢':"pleurer triste ", r'😰':"peur sueur ",
          r'😨':"peur embarasse ",r'😱':"peur crier ",r'😳':"tres surpris embarasse ",r'😵':"tres surpris tres decu ",r'😲':"tres surpris decu ",r'😮':"tres surpris ",
          r'😧':"tres surpris ",r'😦':"tres surpris ",r'😯':"surpris ",r'😑':"neutre ",r'😐':"neutre ", r'😶':"sans commentaire ",r'😡':"trop enerve ",
          r'😠':"enerve ",r'😤':"tres enerve ",r'😩':"se plaindre triste ",r'😫':"se plaindre triste ", r'😖':"impuissance confu ",r'😣':"impuissance ",
          r'☹️':"un peu triste ",r'🙁':"desaccord un peu triste ",r'😕':"desaccord ",r'😟':"desaccord triste ",r'😔':"decu ennui ",
          r'😞':"decu triste ",r'😒':"decu ennui ",r'😓':"decu ",r'😪':"ensommeillé ",r'😴':"dormir ",r'🙄':"ennui ",
          r'🤔':"pensif ",r'❤️':"aimer ",r'😁':"desole ",r'🤐':"gardien du secret ",r'😷':"masque ",r'🤒':"malade fievre ",
          r'🤕':"malade ",r'😈':"mal ",r'👿':"enerver ",r'👺':"enerver ",r'💩':"degoutant ",r'😻':"aimer ",r'🙌':"accord ",
          r'👏':"aplaudissement ",r'🙏':"priere ",r'👍':"bien ",r'👎':"mauvais ",r'👊':"ok ",r'✌️':"victoire ", r'👌':"tres bien",r'💪':"fort ",r'👩‍':"amoureux ",r'👧‍':"amoureux ", r'👦':"content "
         }

for k in D_emojis:
    DataBase['comment']=DataBase['comment'].apply(lambda x: re.sub(k,D_emojis[k],x))

Remplacer les lettres accentuées par des non accentuées

In [103]:
D_Accents={"\xea":"e","\xe9":"e","\xe8":"e","\xe0":"a","\xfb":"u","\xf1":"n","\xf2":"o","\xf3":"o","\xf4":"o","\xf5":"o","\xf6":"o"}
for i in D_Accents.keys():
    DataBase['comment']=DataBase['comment'].apply(lambda x: re.sub(i,D_Accents[i],x))

Eliminer tout les @XXXX et les caractères spéciaux. 

In [104]:
DataBase['comment'] = DataBase['comment'].apply(lambda x: re.sub(r'@([a-zA-Z0-9]*)', '', x))
DataBase['comment'] = DataBase['comment'].apply(lambda x: re.sub(r'[/:\(\)\%+0-9\n_]+', '', x))
DataBase['comment'] = DataBase['comment'].apply(lambda x: re.sub(r'\\([A-Z0-9a-z]+)', '', x))
DataBase['comment'] = DataBase['comment'].apply(lambda x: re.sub(r'[^\w]', ' ', x))
DataBase.head()

,comment,Product,Brand,Date
0,C beau d avoir un smartphone denier cri mais p...,gen_smartphone,NA,2016-12-26
1,alxbrd bloquee sur smartphone gt droits d...,gen_smartphone,NA,2016-12-26
2,aucun Melle un bon Wiko suffit Ras le bol d...,gen_smartphone,NA,2016-12-26
3,moi je veux gagner un smartphone je me suis c...,gen_smartphone,NA,2016-12-26
4,je sais que c est une ville tres culturel mai...,gen_smartphone,NA,2016-12-26


### 2. Traiter les données et préparer la base pour la détection des axes

Garder les substantifs de plus de 3 lettres.

In [105]:
from pattern.fr import parse, split
DataBase['comment_tokens'] = DataBase['comment'].apply(lambda x: parse(x.replace('.',""), chunks = False, lemmata = True).split()[0])
DataBase['comment_lemmas'] = DataBase['comment_tokens'].apply(lambda x: [t[2] for t in x if t[1] in ['NN','NNS','NNPS','NNP'] ])

In [106]:
from gensim.models import Phrases
M_bigram=50
bigram = Phrases(DataBase['comment_lemmas'], min_count=20)
for idx in range(len(DataBase)):
    for token in bigram[DataBase['comment_lemmas'][idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            DataBase['comment_lemmas'][idx].append(token)

DataBase['comment_relevant_Axis']=DataBase['comment_lemmas'].apply(lambda x:  [t for t in x if len(t)> 3])
DataBase['comment_relevant_Axis']=DataBase['comment_relevant_Axis'].apply(lambda x:  " ".join(x))

In [107]:
DataBase.head()

,comment,Product,Brand,Date,comment_tokens,comment_lemmas,comment_relevant_Axis
0,C beau d avoir un smartphone denier cri mais p...,gen_smartphone,NA,2016-12-26,"[[C, JJ, c], [beau, RB, beau], [d, JJ, d], [av...","[smartphone, denier, cri, notif, tre, tre]",smartphone denier notif
1,alxbrd bloquee sur smartphone gt droits d...,gen_smartphone,NA,2016-12-26,"[[alxbrd, NN, alxbrd], [bloquee, NN, bloquee],...","[alxbrd, bloquee, smartphone, gt, droit, auteu...",alxbrd bloquee smartphone droit auteur ordi
2,aucun Melle un bon Wiko suffit Ras le bol d...,gen_smartphone,NA,2016-12-26,"[[aucun, DT, aucun], [Melle, NNP, melle], [un,...","[melle, wiko, ras, bol, societe, gen, smartphone]",melle wiko societe smartphone
3,moi je veux gagner un smartphone je me suis c...,gen_smartphone,NA,2016-12-26,"[[moi, PRP, moi], [je, PRP, je], [veux, VB, vo...","[smartphone, connecte, conte]",smartphone connecte conte
4,je sais que c est une ville tres culturel mai...,gen_smartphone,NA,2016-12-26,"[[je, PRP, je], [sais, VB, savoir], [que, IN, ...","[ville, tre, dieu, merci, statues, smartphone]",ville dieu merci statues smartphone


### 2. Enrechissement de la base: Détection des thématiques:

In [108]:
DataBase.to_csv("CleanedData/Tweets_DataBase.csv", sep='\t', encoding='utf-8')

In [3]:
DataBase=pd.read_csv("CleanedData/DataBase_Axis.csv", sep=str(","), index_col =0)


### 3. Déterminer le score de sentiment et subjectivité des commentaires

A ce niveau nous nous contentons d'utiliser la bibliothèque pattern.fr qui s'avère plus performante (rapide, précise et simple). Les détails concernant les modèles que nous avons développé sont dans les autres notebooks. 

In [4]:
DataBase['comment_sentiments']=DataBase['comment'].apply(lambda x: sentiment(x)[0])
DataBase['comment_objectivity']=DataBase['comment'].apply(lambda x: sentiment(x)[1])

In [5]:
DataBase.head()

,comment,Product,Brande,Date,Thematiques_code,Thematiques,comment_sentiments,comment_objectivity
0,C beau d avoir un smartphone denier cri mais p...,gen_smartphone,NaN,2016-12-26,General,General,0.300,0.50
1,alxbrd bloquee sur smartphone gt droits d...,gen_smartphone,NaN,2016-12-26,General,General,0.170,0.00
2,aucun Melle un bon Wiko suffit Ras le bol d...,gen_smartphone,NaN,2016-12-26,General,General,0.205,0.45
3,moi je veux gagner un smartphone je me suis c...,gen_smartphone,NaN,2016-12-26,General,General,0.220,0.10
4,je sais que c est une ville tres culturel mai...,gen_smartphone,NaN,2016-12-26,General,General,0.250,0.30


## Charger pour la Visualisation

In [7]:
DataBase.to_csv("CleanedData/Tweets_Sentiment_Axis_DataBase.csv", sep=';', encoding='utf-8')